In [1]:
from json import loads
import pandas as pd
from ast import literal_eval
import re

FILE_PATH = "parser_output/Juri_example_output_from_parser.csv"

In [2]:
datas = pd.read_csv(FILE_PATH)
print(datas)

                               name  cmc  \
0         Juri, Master of the Revue  2.0   
1                       Woe Strider  3.0   
2                 Bonecrusher Giant  3.0   
3                             Stomp  2.0   
4         Chandra, Acolyte of Flame  3.0   
..                              ...  ...   
99      Fable of the Mirror-Breaker  3.0   
100         Reflection of Kiki-Jiki  0.0   
101  Sokenzan, Crucible of Defiance  0.0   
102                Voldaren Epicure  1.0   
103            Bloodtithe Harvester  2.0   

                                                  text  mana_cost  card_id  \
0    Whenever you sacrifice a permanent, put a +1/+...     {B}{R}        0   
1    When Woe Strider enters the battlefield, creat...     {2}{B}        1   
2    Whenever Bonecrusher Giant becomes the target ...     {2}{R}        2   
3    Damage can't be prevented this turn. Stomp dea...     {1}{R}        2   
4    0: Put a loyalty counter on each red planeswal...  {1}{R}{R}        3   
.. 

In [3]:
s = datas.iloc[100]

In [4]:
datas.loc[100]

name                                        Reflection of Kiki-Jiki
cmc                                                             0.0
text              {1}, {T}: Create a token that's a copy of anot...
mana_cost                                                       NaN
card_id                                                          96
is_dfc                                                         True
color_identity                                                ['R']
board                                                     mainboard
power                                                             2
toughness                                                         2
supertype                                                        []
type                                    ['Enchantment', 'Creature']
subtype                                        ['Goblin', 'Shaman']
Name: 100, dtype: object

In [5]:
BASIC_LAND_TYPES = ["Swamp", "Mountain", "Island", "Plains", "Forest"]
class Card :
    ########### INSTANT AND SORCERIES CATEGORY #########
    BURN_CATEGORY = "BURN_CATEGORY"
    DRAW_CATEGORY = "DRAW_CATEGORY"
    COUNTER_CATEGORY = "COUNTER_CATEGORY"
    MANA_CATEGORY = "MANA_CATEGORY"
    PUMP_CATEGORY = "PUMP_CATEGORY"
    TUTOR_CATEGORY = "TUTOR_CATEGORY"
    REANIMATOR_CATEGORY = "REANIMATOR_CATEGORY"
    
    
    SUB_REANIMATOR_BATTLEFIELD = "SUB_REANIMATOR_BATTLEFIELD"
    SUB_REANIMATOR_HAND = "SUB_REANIMATOR_HAND"
    
    SUB_TUTOR_HAND = "SUB_TUTOR_HAND"
    SUB_TUTOR_TOP = "SUB_TUTOR_HAND"
    
    ########### LANDS CATEGORY #########
    LAND_TYPE_BASIC = "LAND_TYPE_BASIC"
    LAND_TYPE_FETCH = "LAND_TYPE_FETCH"
    LAND_TYPE_TRUE_DUAL = "LAND_TYPE_TRUE_DUAL"
    LAND_TYPE_THRESHOLD = "LAND_TYPE_THRESHOLD"
    LAND_TYPE_OTHER = "LAND_TYPE_OTHER"
    ########### METHODS #########
    def __init__(self, raw):
        self.name = raw["name"]
        self.cmc = raw["cmc"]
        self.text = raw["text"]
        self.mana_cost = raw["mana_cost"]
        self.is_dfc = raw["is_dfc"]
        self.color_identity = literal_eval(raw["color_identity"])
        self.board = raw["board"]
        self.power = raw["power"]
        self.toughness = raw["toughness"]
        self.supertype = literal_eval(raw["supertype"])
        self.type = literal_eval(raw["type"])
        self.subtype = literal_eval(raw["subtype"])
        self.category = []
        self.subcategory = []
        self.produce_colors = []
        
    def set_is_permanent(self, is_permanent) :
        self.is_permanent = is_permanent
        
    def get_is_permanent(self) :
        if self.is_permanent :
            return "permanent"
        else : 
            return "not permanent"
    def add_category(self, category) :
        self.category.append(category)
    def print_category(self) :
        print("Category : ", self.category)
    def add_subcategory(self, category) :
        self.subcategory.append(category)
    def print_subcategory(self) :
        print("Sub category : ", self.subcategory)
    def add_produce_colors(self, colors) :
        self.produce_colors.append(colors)
    def print_produce_colors(self) :
        print("Can produce : ", self.produce_colors)

In [35]:
def filter_by_type(card) :
    permanent_type = ["Creature", "Planeswalker", "Enchantment", "Artifact", "Land"]
    non_permanent_type = ["Instant", "Sorcery"]
    for card_type in card.type :
        if card_type == "Creature" :
            filter_creature(card)
        if card_type == "Planeswalker" :
            filter_planeswalker(card)
        if card_type == "Enchantment" :
            filter_enchantment(card)
        if card_type == "Artifact" :
            filter_artifact(card)
        if card_type == "Land" :
            filter_land(card)    
        if card_type == "Instant" :
            filter_not_permanent(card)
            filter_instant(card)
        if card_type == "Sorcery" :
            filter_not_permanent(card)
            filter_sorcery(card) 
    
def filter_creature(card) :
    card.set_is_permanent(True)
    if dork_criterion(card) :
        filter_creature_dork(card)
def filter_planeswalker(card) :
    card.set_is_permanent(True)
def filter_enchantment(card) :
    card.set_is_permanent(True)
def filter_artifact(card) :
    card.set_is_permanent(True)
def filter_land(card) :
    card.set_is_permanent(True)
    card.add_category(land_criterion(card))
    if card.LAND_TYPE_FETCH in card.category : 
        card.add_subcategory(land_fetch_type_criterion(card))
    if card.LAND_TYPE_BASIC in card.category :
        colors = find_colors_produced_basic_criterion(card)
        card.add_produce_colors(colors)
        card.add_subcategory(colors)
    if card.LAND_TYPE_TRUE_DUAL in card.category :
        colors = find_colors_produced_basic_criterion(card)
        card.add_produce_colors(colors)
        card.add_subcategory(colors)
    if card.LAND_TYPE_THRESHOLD in card.category : 
        colors = find_colors_produced_advanced_criterion(card)
        card.add_subcategory(colors)
        card.add_produce_colors(colors)
def filter_not_permanent(card) :
    card.set_is_permanent(False)
    if counter_criterion(card) :
        card.add_category(card.COUNTER_CATEGORY)
        pass #Todo 
    if burn_criterion(card) :
        card.add_category(card.BURN_CATEGORY)
        pass
    if draw_criterion(card) :
        card.add_category(card.DRAW_CATEGORY)
        pass #Todo
    if mana_criterion(card) :
        card.add_category(card.MANA_CATEGORY)
        pass #Todo
    if pump_criterion(card) :
        card.add_category(card.PUMP_CATEGORY)
        pass #Todo
    if tutor_criterion(card) :
        card.add_category(card.TUTOR_CATEGORY)
        pass #Todo
    if reanimator_criterion(card) :
        card.add_category(card.REANIMATOR_CATEGORY)
        pass #Todo
def filter_sorcery(card) :
    pass #Todo
def filter_instant(card) :
    pass #Todo
########### INSTANT AND SORCERIES CRITERIONS #########
def counter_criterion(card) :
    return False #TODO
def burn_criterion(card) :
    burn_regex = r"deals [0-9]+ damages* to (any )*target"
    match = re.search(burn_regex, card.text) is not None
    return match
def draw_criterion(card) :
    return False #TODO
def mana_criterion(card) :
    return False #TODO
def pump_criterion(card) :
    return False #TODO
def tutor_criterion(card) :
    tutor_regex = r"Search your library for .* put .*"
    match = re.search(tutor_regex, card.text) is not None
    if match :
        card.add_subcategory(tutor_sub_criterion(card))
    return match
def reanimator_criterion(card) :
    reanimator_regex = r"target creature card from .* graveyard .*to .*"
    match = re.search(reanimator_regex, card.text) is not None
    if match :
        card.add_subcategory(reanimator_sub_criterion(card))
    return match

########### SUB INSTANT AND SORCERIES CRITERIONS #########
def reanimator_sub_criterion(card) :
    battlefield_regex = r"onto the battlefield"
    match = re.search(battlefield_regex, card.text) is not None
    if match :
        return Card.SUB_REANIMATOR_BATTLEFIELD
    hand_regex = r"to your hand"
    match = re.search(hand_regex, card.text) is not None
    if match :
        return Card.SUB_REANIMATOR_HAND
def tutor_sub_criterion(card) :
    top_regex = r"on top of it"
    match = re.search(top_regex, card.text) is not None
    if match :
        return Card.SUB_TUTOR_TOP
    hand_regex = r"to your hand"
    match = re.search(hand_regex, card.text) is not None
    if match :
        return Card.SUB_TUTOR_HAND
########### CREATURES CRITERIONS #########
def dork_criterion(card) :
    return False #TODO

########### LANDS CRITERIONS #########
def land_criterion(card) :
    print(card.supertype)
    if "Basic" in card.supertype :
        return Card.LAND_TYPE_BASIC
    
    true_duals_regex = r"^\({T}: Add {.} or {.}.\)$"
    match = re.search(true_duals_regex, card.text) is not None
    if match :
        return Card.LAND_TYPE_TRUE_DUAL
    
    fetch_regex = r"^{T}, Pay 1 life, Sacrifice .*: Search your library for an .* or .* card, put it onto the battlefield, then shuffle.$"
    match = re.search(fetch_regex, card.text) is not None
    if match :
        return Card.LAND_TYPE_FETCH
    
    threshold_regex = r"Threshold — "
    match = re.search(threshold_regex, card.text) is not None
    if match :
        return Card.LAND_TYPE_THRESHOLD 
    
    return Card.LAND_TYPE_OTHER #TODO
########### LANDS SUB CRITERIONS #########
def land_fetch_type_criterion(card) :
    types = []
    for land_type in BASIC_LAND_TYPES :
        if land_type in card.text :
            types.append(land_type)
    return types
def find_colors_produced_basic_criterion(card) :
    basic_colors_regex = r"{[B,W,G,U,R]}"
    return re.findall (basic_colors_regex, card.text)
def find_colors_produced_advanced_criterion(card) :
    mana_section_regex = r"[aA]dd .*}\."
    mana_section = re.findall(mana_section_regex, card.text)
    mana_section_ = ' '.join(mana_section)
    basic_colors_regex = r"{[B,W,G,U,R]}"
    return re.findall(basic_colors_regex, mana_section_)

In [36]:
spells = [90, 45, 44, 41, 34, 39, 59, 40, 49]
for i in spells :
    print("-----------------")
    card = Card(datas.loc[i])
    # 90 --> bolt
    filter_by_type(card)
    
    print(card.name)
    print(card.get_is_permanent())
    card.print_category()
    card.print_subcategory()
    if  "Land" in card.type :
        card.print_produce_colors() 


-----------------
Lightning Bolt
not permanent
Category :  ['BURN_CATEGORY']
Sub category :  []
-----------------
Reanimate
not permanent
Category :  ['REANIMATOR_CATEGORY']
Sub category :  ['SUB_REANIMATOR_BATTLEFIELD']
-----------------
Chain Lightning
not permanent
Category :  ['BURN_CATEGORY']
Sub category :  []
-----------------
Demonic Tutor
not permanent
Category :  ['TUTOR_CATEGORY']
Sub category :  ['SUB_TUTOR_HAND']
-----------------
Collateral Damage
not permanent
Category :  ['BURN_CATEGORY']
Sub category :  []
-----------------
[]
Barbarian Ring
permanent
Category :  ['LAND_TYPE_THRESHOLD']
Sub category :  [['{R}']]
Can produce :  [['{R}']]
-----------------
['Basic']
Mountain
permanent
Category :  ['LAND_TYPE_BASIC']
Sub category :  [['{R}']]
Can produce :  [['{R}']]
-----------------
[]
Badlands
permanent
Category :  ['LAND_TYPE_TRUE_DUAL']
Sub category :  [['{B}', '{R}']]
Can produce :  [['{B}', '{R}']]
-----------------
[]
Polluted Delta
permanent
Category :  ['LAND_TY